Note: The notes contained in this notebook were taken from DataCamp's "Machine Learning with PySpark"
# Machine Learning with PySpark
Spark is a powerful, general purpose tool for working with Big Data. Spark transparently handles the distribution of compute tasks across a cluster. This means that operations are fast, but it also allows you to focus on the analysis rather than worry about technical details. In this course you'll learn how to get data into Spark and then delve into the three fundamental Spark Machine Learning algorithms: Linear Regression, Logistic Regression/Classifiers, and creating pipelines. Along the way you'll analyse a large dataset of flight delays and spam text messages. With this background you'll be ready to harness the power of Spark and apply it on your own Machine Learning projects!

**Instructor:** Andrew Collier, Data Scientist @ Exegetic Analytics

#### First, some recap:
Spark is a platform for cluster computing. Spark lets you spread data and computations over clusters with multiple nodes (think of each node as a separate computer). Splitting up your data makes it easier to work with very large datasets because each node only works with a small amount of data.

As each node works on its own subset of the total data, it also carries out a part of the total calculations required, so that both data processing and computation are performed in parallel over the nodes in the cluster. It is a fact that parallel computation can make certain types of programming tasks much faster.

However, with greater computing power comes greater complexity.

Deciding whether or not Spark is the best solution for your problem takes some experience, but you can consider questions like:
    * Is my data too big to work with on a single machine?
    * Can my calculations be easily parallelized?
    
The first step in using Spark is connecting to a cluster.

In practice, the cluster will be hosted on a remote machine that's connected to all other nodes. There will be one computer, called the master that manages splitting up the data and the computations. The master is connected to the rest of the computers in the cluster, which are called worker. The master sends the workers data and calculations to run, and they send their results back to the master.

When you're just getting started with Spark it's simpler to just run a cluster locally. Thus, for this course, instead of connecting to another computer, all computations will be run on DataCamp's servers in a simulated cluster.

Creating the connection is as simple as creating an instance of the `SparkContext` class. The class constructor takes a few optional arguments that allow you to specify the attributes of the cluster you're connecting to.

An object holding all these attributes can be created with the `SparkConf()` constructor. Take a look at the [documentation](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html) for all the details!

#### Using DataFrames
Spark's core data structure is the **Resilient Distributed Dataset (RDD)**. This is a low level object that lets Spark work its magic by splitting data across multiple nodes in the cluster. However, RDDs are hard to work with directly, so in this course you'll be using the Spark DataFrame abstraction built on top of RDDs.

The Spark DataFrame was designed to behave a lot like a SQL table (a table with variables in the columns and observations in the rows). Not only are they easier to understand, DataFrames are also more optimized for complicated operations than RDDs.

When you start modifying and combining columns and rows of data, there are many ways to arrive at the same result, but some often take much longer than others. When using RDDs, it's up to the data scientist to figure out the right way to optimize the query, but the DataFrame implementation has much of this optimization built in!

To start working with Spark DataFrames, you first have to create a `SparkSession` object from your `SparkContext`. You can think of the `SparkContext` as your connection to the cluster and the `SparkSession` as your interface with that connection.

#### Put some Spark in your data
In the last exercise, you saw how to move data from Spark to `pandas`. However, maybe you want to go the other direction, and put a `pandas` DataFrame into a Spark cluster! The `SparkSession` class has a method for this as well.

The `.createDataFrame()` method takes a `pandas` DataFrame and returns a Spark DataFrame.

The output of this method is stored locally, not in the `SparkSession` catalog. This means that you can use all the Spark DataFrame methods on it, but you can't access the data in other contexts.

For example, a SQL query (using the `.sql()` method) that references your DataFrame will throw an error. To access the data in this way, you have to save it as a temporary table.

You can do this using the `.createTempView()` Spark DataFrame method, which takes as its only argument the name of the temporary table you'd like to register. This method registers the DataFrame as a table in the catalog, but as this table is temporary, it can only be accessed from the specific `SparkSession` used to create the Spark DataFrame.

There is also the method `.createOrReplaceTempView()`. This safely creates a new temporary table if nothing was there before, or updates an existing table if one was already defined. You'll use this method to avoid running into problems with duplicate tables.

Check out the diagram to see all the different ways your Spark data structures interact with each other.

<img src='data/spark_createTempView.png' width="400" height="200" align="center"/>

* Similar to `.withColumn()`, you can do column-wise computations within a `SELECT` statement. 
* `SELECT origin, dest, air_time / 60 FROM flights;`
* the following two expressions will produce the same output:
* `flights.filter("air_time > 120").show()`
* `flights.filter(flights.air_time > 120).show()`
* The difference between `.select()` and `.withColumn()` methods is that `.select()` returns only the columns you specify, while .`withColumn()` returns all the columns of the DataFrame in addition to the one you defined. 
* At the core of the `pyspark.ml` module are the `Transformer` and `Estimator` classes. Almost every other class in the module behaves similarly to these two basic classes.
* `Transformer` classes have a `.transform()` method that takes a DataFrame and returns a new DataFrame; usually the original one with a new column appended. For example, you might use the class `Bucketizer` to create discrete bins from a continuous feature or the class `PCA` to reduce the dimensionality of your dataset using principal component analysis.
* `Estimator` classes all implement a `.fit()` method. These methods also take a DataFrame, but instead of returning another DataFrame they return a model object. This can be something like a `StringIndexerModel` for including categorical data saved as strings in your models, or a `RandomForestModel` that uses the random forest algorithm for classification or regression.
* Before you get started modeling, it's important to know that Spark only handles numeric data. That means all of the columns in your DataFrame must be either integers or decimals (called 'doubles' in Spark).
* To remedy this, you can use the `.cast()` method in combination with the `.withColumn()` method. It's important to note that `.cast()` works on columns, while `.withColumn()` works on DataFrames.
* In Spark it's important to make sure you split the data **after** all the transformations. This is because operations like `StringIndexer` don't always produce the same index even when given the same list of strings.

# $\star$ Chapter 1: Introduction

## Machine Learning and Spark
Spark is a framework for working with Big Data. In this chapter you'll cover some background about Spark and Machine Learning. You'll then find out how to connect to Spark using Python and load CSV data.

Here, we'll learn how to build Machine Learning models on large data sets using distributed computing techniques

* The performance of an ML depends on data; in general, more data is a good thing
* If the data can fit entirely in RAM then the algorithm can operate efficiently
* When the data no longer fit into memory, the computer will start to use **virutal memory** and the data will be **paged** back and forth between RAM and disk
    * Relative to **RAM** access, retrieving data from disk is slow
    * And as the size of the data grows, paging becomes more intense and the computer begins to spend more and more time waiting for data. Performance plummets.

<img src='data/datasize_RAM.png' width="600" height="300" align="center"/>

* One option is to **distribute the problem across multiple computers in a cluster;** rather than trying to handle a large dataset on a single machine, it's divided up into partitions which are processed separately.
    * Ideally each data partition can fit into RAM on a single computer in the cluster.
    * This is the approach used by **Spark**
    
#### What is Spark?
* Compute across a distributed cluster
* Data processing in memory
* Well-documented high-level API
* **It is generally much faster than other Big Data technologies like Hadoop, because it does most processing in memory .**
* **It has a developer-friendly interface which hides much of the complexity of distributed computing.**

#### Cluster components
* A cluster consists of one or more **nodes**
* Each **node** is a computer with CPU, RAM, and physical storage
* A **cluster manager** allocates resources and coordinates activity across the cluster
* Every application running on the Spark cluster has a **driver** program
* Using the **Spark API**, the driver communicates with the cluster manager, which in turn distributes work to the nodes.
* On each node, Spark launches an **executor** process which persists for the duration of the application
* Work is divided up into **tasks**, which are simply units of computation
* The executors run tasks in multiple **threads** across the **cores** in a node

<img src='data/cluster_structure.png' width="500" height="250" align="center"/>

<img src='data/course_datasets.png' width="600" height="300" align="center"/>